In [44]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils
import keras
# from https://github.com/RohitSaha/VGG_Imagenet_Weights_GrayScale_Images/blob/master/convert_vgg_grayscale.py
from keras.applications.vgg16 import VGG16
from keras.models import load_model, Model
from keras.layers import Input, Conv2D, MaxPooling2D

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import sys
from PIL import Image
from easydict import EasyDict

In [76]:
# DO NOT CHANGE-------------------------------------------------------------------------------------------

# creates new weights for a 1 channel image
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
model = VGG16(weights='imagenet', include_top=False)

# Block1_conv1 weights are of the format [3, 3, 3, 64] -> this is for RGB images
# For grayscale, format should be [3, 3, 1, 64]. Weighted average of the features has to be calculated across channels.
# RGB weights: Red 0.2989, Green 0.5870, Blue 0.1140

# getting weights of block1 conv1.
block1_conv1 = model.get_layer('block1_conv1').get_weights()
weights, biases = block1_conv1

# :weights shape = [3, 3, 3, 64] - (0, 1, 2, 3)
# convert :weights shape to = [64, 3, 3, 3] - (3, 2, 0, 1)
weights = np.transpose(weights, (3, 2, 0, 1))


kernel_out_channels, kernel_in_channels, kernel_rows, kernel_columns = weights.shape

# Dimensions : [kernel_out_channels, 1 (since grayscale), kernel_rows, kernel_columns]
grayscale_weights = np.zeros((kernel_out_channels, 1, kernel_rows, kernel_columns))

# iterate out_channels number of times
for i in range(kernel_out_channels):

	# get kernel for every out_channel
	get_kernel = weights[i, :, :, :]

	temp_kernel = np.zeros((3, 3))

	# :get_kernel shape = [3, 3, 3]
	# axis, dims = (0, in_channel), (1, row), (2, col)

	# calculate weighted average across channel axis
	in_channels, in_rows, in_columns = get_kernel.shape

	for in_row in range(in_rows):
		for in_col in range(in_columns):
			feature_red = get_kernel[0, in_row, in_col]
			feature_green = get_kernel[1, in_row, in_col]
			feature_blue = get_kernel[2, in_row, in_col]

			# weighted average for RGB filter
			total = (feature_red * 0.2989) + (feature_green * 0.5870) + (feature_blue * 0.1140)

			temp_kernel[in_row, in_col] = total


	# :temp_kernel is a 3x3 matrix [rows x columns]
	# add an axis at the end to specify in_channel as 1

	# 2 ways of doing this,

	# First: Add axis directly at the end of :temp_kernel to make its shape: [3, 3, 1], but this might be 
	# an issue when concatenating all feature maps

	# Second: Add axis at the start of :temp_kernel to make its shape: [1, 3, 3] which is [in_channel, rows, columns]
	temp_kernel = np.expand_dims(temp_kernel, axis=0)

	# Now, :temp_kernel shape is [1, 3, 3]

	# Concat :temp_kernel to :grayscale_weights along axis=0
	grayscale_weights[i, :, :, :] = temp_kernel

# Dimension of :grayscale_weights is [64, 1, 3, 3]
# In order to bring it to tensorflow or keras weight format, transpose :grayscale_weights

# dimension, axis of :grayscale_weights = (out_channels: 0), (in_channels: 1), (rows: 2), (columns: 3)
# tf format of weights = (rows: 0), (columns: 1), (in_channels: 2), (out_channels: 3)

# Go from (0, 1, 2, 3) to (2, 3, 1, 0)
grayscale_weights = np.transpose(grayscale_weights, (2, 3, 1, 0)) # (3, 3, 1, 64)

# combine :grayscale_weights and :biases
new_block1_conv1 = [grayscale_weights, biases]


# Reconstruct the layers of VGG16 but replace block1_conv1 weights with :grayscale_weights

# get weights of all the layers starting from 'block1_conv2'
vgg16_weights = {}
for layer in model.layers[2:]:
	if "conv" in layer.name:
		vgg16_weights["200_" + layer.name] = model.get_layer(layer.name).get_weights()

del model


# Custom build VGG16
input = Input(shape=(200, 200, 1), name='200_input')
# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(200, 200, 1), data_format="channels_last", name='200_block1_conv1')(input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='200_block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='200_block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='200_block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='200_block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='200_block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='200_block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='200_block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='200_block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='200_block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='200_block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block5_conv3')(x)
x = MaxPooling2D((8, 8), strides=(8, 8), name='200_block5_pool')(x)

base_model = Model(inputs=input, outputs=x)

base_model.get_layer('200_block1_conv1').set_weights(new_block1_conv1)
for layer in base_model.layers[2:]:
	if 'conv' in layer.name:
		base_model.get_layer(layer.name).set_weights(vgg16_weights[layer.name])

#base_model.summary()

#print base_model.get_layer('block3_conv2').get_weights()
base_model.save('vgg_grayscale_200.hdf5')

# Change {args.dataset, np.array colors}

In [30]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", type=str, default = 'Batches/testing',
    help="path to input dataset")
ap.add_argument("-e", "--epochs", type=int, default=25,
    help="# of epochs to train our network for")
ap.add_argument("-p", "--plot", type=str, default="plot.png",
    help="path to output loss/accuracy plot")
args = args = EasyDict({
    "dataset": 'Batches/testing',
    "epochs": 3,
    'plot':'plot.png'
})

In [64]:
print (len([name for name in os.listdir('Batches/testing/Iffy')]) + \
        len([name for name in os.listdir('Batches/testing/Senescent')]) + \
        len([name for name in os.listdir('Batches/testing/Prolif')]))

3001


In [35]:
im = Image.open('Batches/testing/Iffy/cell1.tif')
imarray = np.array(im)

print('size of cell: 200 x 200 = ',imarray.size)
print('max value', np.amax(imarray))

size of cell: 200 x 200 =  40000
max value 255


In [52]:
# finds out how many channels
# -------------------------------------------

# name of your image file
# filename = 'Batches/testing/Iffy/f.png'
cellname = 'Batches/testing\Iffy\cell1.tif'

# open image using PIL
img = Image.open(cellname)

# convert to numpy array
imgcell = np.array(img)


#---------------useless------------------
# find number of channels
# if imgcell.ndim == 2:
#     channels = 1
#     print("image has 1 channel")
# else:
#     channels = imgcell.shape[-1]
#     print("image has", channels, "channels")
#----------------------------------------------------------
print('raw imgcell: ',imgcell)
print('imgcell shape: ',imgcell.shape)

img = cv2.imread('Batches/testing\Iffy\cell1.tif')
image = np.array(img)
image.shape

raw imgcell:  [[159 157 157 ... 161 161 161]
 [160 160 158 ... 161 161 161]
 [160 160 159 ... 160 161 160]
 ...
 [156 157 157 ... 149 146 150]
 [157 158 156 ... 148 147 151]
 [157 158 159 ... 149 148 156]]
imgcell shape:  (200, 200)


(200, 200, 3)

In [98]:
cell = cv2.imread('Batches/testing\Iffy\cell1.tif',cv2.IMREAD_UNCHANGED)
print(np.shape(cell))
cell = np.expand_dims(cell, axis = 2)
print(type(cell))
cell.shape

(200, 200)
<class 'numpy.ndarray'>


(200, 200, 1)

In [116]:
cell = Image.open('Batches/testing\Iffy\yes.png')
cell = cell.convert(mode = 'L')
np.array(cell).shape

(174, 130)

In [117]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"]))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:

    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    # load the image, swap color channels, and resize it to be a fixed
    # 128x128 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)
# ------------------------------hto garbage---------------
#     
#     print('type', type(imagePath))
#     print(type(image))
#     #image = Image.open(imagePath)
#     print('image shape: ', np.array(image).shape)
#     print('images dimesnions: ',image.ndim)
#     #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (200, 200))
#     print('image dimensions: ', np.array(image).shape)
# ---------------------------------------------------------------
    print(np.shape(image))
    image = np.expand_dims(image, axis = 2)
    # update the data and labels lists, respectively
    print(np.shape(image))
#-------------------------------------------------------------------------------------------------------------------------------
#     img = Image.open(imagePath)
#     image = np.array(img)

#------------------------------------------------------------------------------------------------------------------------------
    data.append(image)
    print(np.shape(data))
    #sys.exit(0)
    labels.append(label)
#np.shape(data)


[INFO] loading images...
(200, 200)
(200, 200, 1)
(1, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2, 200, 200, 1)
(200, 200)
(200, 200, 1)
(3, 200, 200, 1)
(200, 200)
(200, 200, 1)
(4, 200, 200, 1)
(200, 200)
(200, 200, 1)
(5, 200, 200, 1)
(200, 200)
(200, 200, 1)
(6, 200, 200, 1)
(200, 200)
(200, 200, 1)
(7, 200, 200, 1)
(200, 200)
(200, 200, 1)
(8, 200, 200, 1)
(200, 200)
(200, 200, 1)
(9, 200, 200, 1)
(200, 200)
(200, 200, 1)
(10, 200, 200, 1)
(200, 200)
(200, 200, 1)
(11, 200, 200, 1)
(200, 200)
(200, 200, 1)
(12, 200, 200, 1)
(200, 200)
(200, 200, 1)
(13, 200, 200, 1)
(200, 200)
(200, 200, 1)
(14, 200, 200, 1)
(200, 200)
(200, 200, 1)
(15, 200, 200, 1)
(200, 200)
(200, 200, 1)
(16, 200, 200, 1)
(200, 200)
(200, 200, 1)
(17, 200, 200, 1)
(200, 200)
(200, 200, 1)
(18, 200, 200, 1)
(200, 200)
(200, 200, 1)
(19, 200, 200, 1)
(200, 200)
(200, 200, 1)
(20, 200, 200, 1)
(200, 200)
(200, 200, 1)
(21, 200, 200, 1)
(200, 200)
(200, 200, 1)
(22, 200, 200, 1)
(200, 200)
(200, 200, 1)
(23, 200, 200

(200, 200)
(200, 200, 1)
(215, 200, 200, 1)
(200, 200)
(200, 200, 1)
(216, 200, 200, 1)
(200, 200)
(200, 200, 1)
(217, 200, 200, 1)
(200, 200)
(200, 200, 1)
(218, 200, 200, 1)
(200, 200)
(200, 200, 1)
(219, 200, 200, 1)
(200, 200)
(200, 200, 1)
(220, 200, 200, 1)
(200, 200)
(200, 200, 1)
(221, 200, 200, 1)
(200, 200)
(200, 200, 1)
(222, 200, 200, 1)
(200, 200)
(200, 200, 1)
(223, 200, 200, 1)
(200, 200)
(200, 200, 1)
(224, 200, 200, 1)
(200, 200)
(200, 200, 1)
(225, 200, 200, 1)
(200, 200)
(200, 200, 1)
(226, 200, 200, 1)
(200, 200)
(200, 200, 1)
(227, 200, 200, 1)
(200, 200)
(200, 200, 1)
(228, 200, 200, 1)
(200, 200)
(200, 200, 1)
(229, 200, 200, 1)
(200, 200)
(200, 200, 1)
(230, 200, 200, 1)
(200, 200)
(200, 200, 1)
(231, 200, 200, 1)
(200, 200)
(200, 200, 1)
(232, 200, 200, 1)
(200, 200)
(200, 200, 1)
(233, 200, 200, 1)
(200, 200)
(200, 200, 1)
(234, 200, 200, 1)
(200, 200)
(200, 200, 1)
(235, 200, 200, 1)
(200, 200)
(200, 200, 1)
(236, 200, 200, 1)
(200, 200)
(200, 200, 1)
(237, 2

(200, 200)
(200, 200, 1)
(407, 200, 200, 1)
(200, 200)
(200, 200, 1)
(408, 200, 200, 1)
(200, 200)
(200, 200, 1)
(409, 200, 200, 1)
(200, 200)
(200, 200, 1)
(410, 200, 200, 1)
(200, 200)
(200, 200, 1)
(411, 200, 200, 1)
(200, 200)
(200, 200, 1)
(412, 200, 200, 1)
(200, 200)
(200, 200, 1)
(413, 200, 200, 1)
(200, 200)
(200, 200, 1)
(414, 200, 200, 1)
(200, 200)
(200, 200, 1)
(415, 200, 200, 1)
(200, 200)
(200, 200, 1)
(416, 200, 200, 1)
(200, 200)
(200, 200, 1)
(417, 200, 200, 1)
(200, 200)
(200, 200, 1)
(418, 200, 200, 1)
(200, 200)
(200, 200, 1)
(419, 200, 200, 1)
(200, 200)
(200, 200, 1)
(420, 200, 200, 1)
(200, 200)
(200, 200, 1)
(421, 200, 200, 1)
(200, 200)
(200, 200, 1)
(422, 200, 200, 1)
(200, 200)
(200, 200, 1)
(423, 200, 200, 1)
(200, 200)
(200, 200, 1)
(424, 200, 200, 1)
(200, 200)
(200, 200, 1)
(425, 200, 200, 1)
(200, 200)
(200, 200, 1)
(426, 200, 200, 1)
(200, 200)
(200, 200, 1)
(427, 200, 200, 1)
(200, 200)
(200, 200, 1)
(428, 200, 200, 1)
(200, 200)
(200, 200, 1)
(429, 2

(200, 200)
(200, 200, 1)
(596, 200, 200, 1)
(200, 200)
(200, 200, 1)
(597, 200, 200, 1)
(200, 200)
(200, 200, 1)
(598, 200, 200, 1)
(200, 200)
(200, 200, 1)
(599, 200, 200, 1)
(200, 200)
(200, 200, 1)
(600, 200, 200, 1)
(200, 200)
(200, 200, 1)
(601, 200, 200, 1)
(200, 200)
(200, 200, 1)
(602, 200, 200, 1)
(200, 200)
(200, 200, 1)
(603, 200, 200, 1)
(200, 200)
(200, 200, 1)
(604, 200, 200, 1)
(200, 200)
(200, 200, 1)
(605, 200, 200, 1)
(200, 200)
(200, 200, 1)
(606, 200, 200, 1)
(200, 200)
(200, 200, 1)
(607, 200, 200, 1)
(200, 200)
(200, 200, 1)
(608, 200, 200, 1)
(200, 200)
(200, 200, 1)
(609, 200, 200, 1)
(200, 200)
(200, 200, 1)
(610, 200, 200, 1)
(200, 200)
(200, 200, 1)
(611, 200, 200, 1)
(200, 200)
(200, 200, 1)
(612, 200, 200, 1)
(200, 200)
(200, 200, 1)
(613, 200, 200, 1)
(200, 200)
(200, 200, 1)
(614, 200, 200, 1)
(200, 200)
(200, 200, 1)
(615, 200, 200, 1)
(200, 200)
(200, 200, 1)
(616, 200, 200, 1)
(200, 200)
(200, 200, 1)
(617, 200, 200, 1)
(200, 200)
(200, 200, 1)
(618, 2

(200, 200)
(200, 200, 1)
(786, 200, 200, 1)
(200, 200)
(200, 200, 1)
(787, 200, 200, 1)
(200, 200)
(200, 200, 1)
(788, 200, 200, 1)
(200, 200)
(200, 200, 1)
(789, 200, 200, 1)
(200, 200)
(200, 200, 1)
(790, 200, 200, 1)
(200, 200)
(200, 200, 1)
(791, 200, 200, 1)
(200, 200)
(200, 200, 1)
(792, 200, 200, 1)
(200, 200)
(200, 200, 1)
(793, 200, 200, 1)
(200, 200)
(200, 200, 1)
(794, 200, 200, 1)
(200, 200)
(200, 200, 1)
(795, 200, 200, 1)
(200, 200)
(200, 200, 1)
(796, 200, 200, 1)
(200, 200)
(200, 200, 1)
(797, 200, 200, 1)
(200, 200)
(200, 200, 1)
(798, 200, 200, 1)
(200, 200)
(200, 200, 1)
(799, 200, 200, 1)
(200, 200)
(200, 200, 1)
(800, 200, 200, 1)
(200, 200)
(200, 200, 1)
(801, 200, 200, 1)
(200, 200)
(200, 200, 1)
(802, 200, 200, 1)
(200, 200)
(200, 200, 1)
(803, 200, 200, 1)
(200, 200)
(200, 200, 1)
(804, 200, 200, 1)
(200, 200)
(200, 200, 1)
(805, 200, 200, 1)
(200, 200)
(200, 200, 1)
(806, 200, 200, 1)
(200, 200)
(200, 200, 1)
(807, 200, 200, 1)
(200, 200)
(200, 200, 1)
(808, 2

...

(200, 200)
(200, 200, 1)
(2089, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2090, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2091, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2092, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2093, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2094, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2095, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2096, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2097, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2098, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2099, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2100, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2101, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2102, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2103, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2104, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2105, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2106, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2107, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2108, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2109, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2110, 200, 200, 1)
(200, 200)

(2272, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2273, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2274, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2275, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2276, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2277, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2278, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2279, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2280, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2281, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2282, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2283, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2284, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2285, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2286, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2287, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2288, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2289, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2290, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2291, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2292, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2293, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2294, 200

(2456, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2457, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2458, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2459, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2460, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2461, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2462, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2463, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2464, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2465, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2466, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2467, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2468, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2469, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2470, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2471, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2472, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2473, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2474, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2475, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2476, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2477, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2478, 200

(2641, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2642, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2643, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2644, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2645, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2646, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2647, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2648, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2649, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2650, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2651, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2652, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2653, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2654, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2655, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2656, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2657, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2658, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2659, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2660, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2661, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2662, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2663, 200

(200, 200)
(200, 200, 1)
(2825, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2826, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2827, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2828, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2829, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2830, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2831, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2832, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2833, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2834, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2835, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2836, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2837, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2838, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2839, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2840, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2841, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2842, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2843, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2844, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2845, 200, 200, 1)
(200, 200)
(200, 200, 1)
(2846, 200, 200, 1)
(200, 200)

In [118]:
np.shape(data)

(3000, 200, 200, 1)

In [119]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)
print(labels)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labelss = lb.fit_transform(labels)
print(labelss)
print('hi')
#labelss = np_utils.to_categorical(labelss)
print(len(labelss))
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labelss,test_size=0.25, stratify=labelss, random_state=42)

['Iffy' 'Iffy' 'Iffy' ... 'Senescent' 'Senescent' 'Senescent']
[[1 0 0]
 [1 0 0]
 [1 0 0]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]
hi
3000


In [71]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
onehot_encoded
#len(onehot_encoded)


C:\Users\bryan\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [120]:
trainX.shape

(2250, 200, 200, 1)

In [121]:
# initialize the training data augmentation object

#-------------------------------------------------------------------------------------------------------------------------------
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
#-------------------------------------------------------------------------------------------------------------------------------

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects

#-------------------------------------------------------------------------------------------------------------------------------
# mean = np.array([123.68, 116.779, 103.939], dtype="float32")
# trainAug.mean = mean
# valAug.mean = mean
#-------------------------------------------------------------------------------------------------------------------------------


In [122]:
# load VGG16, ensuring the head FC layer sets are left off, while at
# the same time adjusting the size of the input image tensor to the
# network

#-------------------------------------------------------------------------------------------------------------------------------
# baseModel = VGG16(weights="imagenet", include_top=False,
# 	input_tensor=Input(shape=(200, 200, 1)))
#-------------------------------------------------------------------------------------------------------------------------------

#-------------------------------------------------------------------------------------------------------------------------------
baseModel = keras.applications.vgg16.VGG16(
        include_top=False,
        weights='vgg_grayscale_200.hdf5',
        input_tensor=None,
        input_shape=(200, 200, 1),
        pooling=None,
        classes=3)
#-------------------------------------------------------------------------------------------------------------------------------

# show a summary of the base model
print("[INFO] summary for base model...")
print(baseModel.summary())

[INFO] summary for base model...
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 200, 200, 1)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)  

In [123]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(200, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [124]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 200, 200, 1)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0   

In [ ]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = Adam(lr=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=100),
	steps_per_epoch=2,
	validation_data=valAug.flow(testX, testY),
	validation_steps=10,
	epochs=args["epochs"])

[INFO] compiling model...
[INFO] training head...
Epoch 1/3
2/2 [==============================] - 166s 83s/step - loss: 2.0356 - accuracy: 0.3500 - val_loss: 1.3457 - val_accuracy: 0.3281
Epoch 2/3
2/2 [==============================] - 118s 59s/step - loss: 2.2358 - accuracy: 0.3100 - val_loss: 1.3600 - val_accuracy: 0.3688
Epoch 3/3
1/2 [==============>...............] - ETA: 30s - loss: 2.0530 - accuracy: 0.3200

In [81]:
trainX.shape

(2250,)